In [1]:
from smtplib import SMTP_SSL
from email.message import EmailMessage
import glob

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

import password

cred = credentials.Certificate(glob.glob("./aeapolimiweb-firebase*.json")[0])
firebase_admin.initialize_app(cred)

db = firestore.client()

In [2]:
messaggio = """
English version below.

Buongiorno,
Vi segnalo il prossimo evento di AEA: AEAround The World.
Questa volta alcuni studenti di Automazione di recente tornati da Erasmus e doppie lauree in varie parti del mondo
ci racconteranno la loro esperienza, in particolare per quanto riguarda i corsi di Automazione.

L'evento sarà su Zoom, alle 21,30 di martedì 22 dicembre.

Potete iscrivervi al seguente link: https://forms.office.com/Pages/ResponsePage.aspx?id=K3EXCvNtXUKAjjCd8ope69qLTtfc8tBBrOc094iY8J1UQzg0WVpaNUxLVkFYMk4yQU4zVVNUQUVBTC4u

Buone feste,
Team AEA
www.aeapolimi.it


Dear,
Our next event will be AEAround The World.
This time Automation students who recently returned from Erasmus or double degrees will tell us their experience
focusing on what concerns Automation.
The event will be held in Italian (just because it's unlikely that a non-italian speaker in Milan wants to go on Erasmus),
on Zoom, the 22nd of December at 21.30.

You can register at the following link: https://forms.office.com/Pages/ResponsePage.aspx?id=K3EXCvNtXUKAjjCd8ope69qLTtfc8tBBrOc094iY8J1UQzg0WVpaNUxLVkFYMk4yQU4zVVNUQUVBTC4u

Happy Holidays,
AEA Team
www.aeapolimi.it
"""

In [3]:
def main():
    users_ref = db.collection(u'utenti').where(u"newsletter", "==", True)
    docs = users_ref.stream()
    destinatari = [doc.to_dict()["email"] for doc in docs]

    s = SMTP_SSL(host='smtp.office365.com', port=587)
    s.login(password.mail, password.password)
    msg = EmailMessage()
    msg.set_content(messaggio)
    msg['Subject'] = 'AEAround The World'
    msg['From'] = password.mail
    msg['Bcc'] = ", ".join(destinatari)
    s.send_message(msg)
    s.quit()